In [62]:
from mongoengine import *
import os
import pandas as pd
from pandas import Series,DataFrame

使用** mongoengine **  
[connect](http://docs.mongoengine.org/guide/connecting.html#guide-connecting)进行数据库连接

In [2]:
connect('typhoon')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

## 1 读取文件

### 读取方式1：使用pandas

In [15]:
targetfilename=r"5622.txt"
# 家中mac
# targetpath=r"/Users/casablanca/03project/typhoonSearchSys/demo_data"
# 单位mac
targetpath=r"/Users/liusihan/Documents/01project/TyphoonSearchSys/demo_data"
fullname=os.path.join(targetpath,targetfilename)

In [16]:
fullname

'/Users/liusihan/Documents/01project/TyphoonSearchSys/demo_data/5622.txt'

不使用pandas的read_table

In [17]:
with open(fullname,'rb') as f:
    data=pd.read_table(f,sep='\n',encoding='utf-8',header=None,infer_datetime_format=False)
    print('读取成功')

读取成功


In [18]:
data.head(8)

,0
0,1
1,S.N. 122 5622 9 4 QI...
2,0 1 2 3 4 5 6 7 8 9 ...
3,+ ...
4,9 159 156 135 107 90 93 110 123 145 141 ...
5,4 121 109 90 71 58 53 59 73 87 96 ...
6,30 38 47 45 36 32 40 51 50 58 45 ...
7,9 156 159 142 126 106 100 98 -- -- -- ...


In [20]:
# 获取总共数据的长度
len(data)

329

In [107]:
# 找到标志位
mark_list=[]
for index in range(len(data)):
    temp= data.iloc[index][0]
#     print(temp)
    if temp=='1':
        mark_list.append(index)

In [108]:
mark_list

[0, 18, 69, 104, 139, 224, 259, 294]

-----

In [36]:
data.iloc[0][0]

'1'

In [23]:
data.iloc[0][0]

'1'

In [24]:
len(data.iloc[1])

1

In [28]:
data.iloc[1].index

Int64Index([0], dtype='int64')

In [30]:
# data.iloc[1]

In [25]:
data.iloc[1]

0            S.N.   122      5622      9   4     QI...
Name: 1, dtype: object

In [32]:
data.iloc[1].size

1

In [33]:
data.iloc[1][0]

'        S.N.   122      5622      9   4     QINHUANG            Lon  11937   Lat  3955    Lev    89    JW   220    H.G.Y.1975'

In [31]:
type(data.iloc[1])

pandas.core.series.Series

In [54]:
print(type(data.iloc[0][0]))
print(data.iloc[0][0])

<class 'numpy.int64'>
66666


In [55]:
data.iloc[0][0]==66666

True

--------

### 2 获取截取的内容

In [40]:
mark_list

[0, 18, 69, 104, 139, 224, 259, 294]

In [65]:
# 先根据标志位截取一段df
data.iloc[0:17]

,0
0,1
1,S.N. 122 5622 9 4 QI...
2,0 1 2 3 4 5 6 7 8 9 ...
3,+ ...
4,9 159 156 135 107 90 93 110 123 145 141 ...
5,4 121 109 90 71 58 53 59 73 87 96 ...
6,30 38 47 45 36 32 40 51 50 58 45 ...
7,9 156 159 142 126 106 100 98 -- -- -- ...
8,5 130 129 115 95 77 67 65 74 89 101 ...
9,HM 26 30 27 31 29 33 33 -- -- -- ...


对于截取的内容  
index：0：基本信息（包含经纬度信息）

### 2.1 获取台风编号-测站name-经纬度信息

In [69]:
data_base=Series(data.iloc[1][0].split())
data_base

0           S.N.
1            122
2           5622
3              9
4              4
5       QINHUANG
6            Lon
7          11937
8            Lat
9           3955
10           Lev
11            89
12            JW
13           220
14    H.G.Y.1975
dtype: object

In [73]:
# 注意此处需要对经纬度做一个处理（后2位为小数位）

In [74]:
data_base[7][:-2]

'119'

In [75]:
data_base[7][-2:]

'37'

In [76]:
#拼接成小数
float(f"{data_base[7][:-2]}.{data_base[7][-2:]}")

119.37

In [77]:
# 先获取台风编号-测站名称-经纬度信息
name=data_base[5]
code=data_base[2]
latlon=[float(f"{data_base[9][:-2]}.{data_base[9][-2:]}"),float(f"{data_base[7][:-2]}.{data_base[7][-2:]}")]

In [78]:
print(name)
print(code)
print(latlon)

QINHUANG
5622
[39.55, 119.37]


In [47]:
data.iloc[2][0]

'        0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23'

In [56]:
type(data.iloc[2][0])

str

In [59]:
data.iloc[2][0].split()[:5]

['0', '1', '2', '3', '4']

In [79]:
# 转成series
Series(data.iloc[2][0].split())

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
20    20
21    21
22    22
23    23
dtype: object

### 2.2 获取'+'标志位（不用找了）位置固定

In [83]:
data.iloc[3][0].split()[0]

'+'

### 2.3 获取极大值序列与极小值序列  
        差值不要

In [92]:
max_arr=Series(data.iloc[4][0].split())
max_arr.head()

0      9
1    159
2    156
3    135
4    107
dtype: object

In [89]:
# 9月4日的 24小时极大值序列
max_arr[1:25]

1     159
2     156
3     135
4     107
5      90
6      93
7     110
8     123
9     145
10    141
11    155
12    163
13    166
14    175
15    165
16    154
17    157
18    185
19    203
20    204
21    197
22    184
23    160
24    149
dtype: object

In [93]:
# 极小值序列
min_arr=Series(data.iloc[5][0].split())
min_arr.head()

0      4
1    121
2    109
3     90
4     71
dtype: object

In [95]:
# 获取24小时的极小值序列
min_arr[1:25]

1     121
2     109
3      90
4      71
5      58
6      53
7      59
8      73
9      87
10     96
11    106
12    121
13    136
14    140
15    135
16    130
17    132
18    139
19    147
20    154
21    156
22    148
23    135
24    128
dtype: object

### 2.3 获取共有几日的观测数据

In [96]:
len(data.iloc[0:17])

17

In [98]:
data.iloc[0:17].iloc[10]

0                                                  ...
Name: 10, dtype: object

In [99]:
data.iloc[0:17].iloc[9]

0     HM    26  30  27  31  29  33  33  --  --  -- ...
Name: 9, dtype: object

In [100]:
data.iloc[0:17].iloc[14]

0                                                  ...
Name: 14, dtype: object

In [101]:
data.iloc[0:17].iloc[14][0]

'                                                                                                                                      MIN     '

In [103]:
data.iloc[0:17].iloc[14][0].split()

['MIN']

In [102]:
data.iloc[0:17].iloc[10][0]

'                                                                                                  MAX                                         '

In [104]:
data.iloc[0:17].iloc[10][0].split()

['MAX']

----

## 3 封装成方法  
- 获取所有的需要截取的标志位（即1出现的位置）  
- 根据窃取的标志位进行按日进行截取（此处需要判断 该站位，对应的台风 共有几日的观测数据）